In [ ]:
# Scraping from Filets bleus website eclate au sole

In [ ]:
# Needed ?
cd .

/var/www/kerlandrier_mono/api/script/scraping/2025_filets_bleus


In [ ]:
# Import libs
import sys
import os
import git
import pandas as pd
from dateparser import parse as dparse

# Ajoute le dossier "ressources" au sys.path
git_root = git.Repo(search_parent_directories=True).working_tree_dir
sys.path.insert(0,   os.path.abspath(  os.path.join(  git_root,'api' ) ) )

import script.libs.utils as utils
import script.libs.scraping_utils as scraping_utils
import script.libs.HttpRequests as HttpRequests
from script.configuration import config, oa
from script.libs.getOaLocation import get_or_create_oa_location

from slugify import slugify
import os
import requests
# import requests_cache
from bs4 import BeautifulSoup
import  dateparser, pytz
from urllib.parse import urlparse

In [ ]:
# Constants for Scraping
baseURL= "https://www.festivaldesfiletsbleus.bzh/"
allEventsPage= f"{baseURL}/programme_filets_bleus_2025.html"
# locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
# DEFAULT_TIME_START = "20:00:00"
# DEFAULT_TIME_END = "22:00:00"
getTimeout= scraping_utils.getTimeout

In [ ]:
# Get the HTML content of the events page
html_doc = requests.get(allEventsPage, timeout=getTimeout).content.decode('ISO-8859-1',errors='replace')
parsed_html = BeautifulSoup(html_doc ,'html.parser')

In [ ]:
# Extract concerts dates (3) from the HTML
dateSelector= "div > h6"
extractedDates= extractedConcerts = parsed_html.select(dateSelector)
dates = [f"{d.text} 2025" for d in extractedDates if d.text.strip() != ""]
print("Dates:", dates)

Dates: ['MERCREDI 13 AOÃ\x9bT 2025', 'JEUDI 14 AOÃ\x9bT 2025', 'VENDREDI 15 AOÃ\x9bT 2025', 'SAMEDI 16 AOÃ\x9bT 2025', 'DIMANCHE 17 AOÃ\x9bT 2025']


In [ ]:
import pprint

# Extract timing and names
allEventsSelector= "span.PROGTXT p" #encarts où cse trouve la liste des événements
for e in parsed_html.find_all('br'):
    e.extract()
extractedConcerts = parsed_html.select(allEventsSelector)

# Split concerts start,end and names in List
concerts=[]
for d,part in enumerate(extractedConcerts):
    splitted =part.get_text().strip().split('\n')
    i=0
    for c in splitted:
        # Merging lines from the same event
        if c.find(":") <0 and i > 0: 
            concerts[-1] = f"{(splitted[i-1]+ " " + splitted[i]).replace(":","__")} __ {dates[d]} __ {baseURL}{part.select("a")[i-1]["href"].strip()}"
            c=c.replace(":","__")
            continue
        c=c.replace(":","__")
        concerts.append( f"{c.strip()} __ {dates[d]} __ {baseURL}{part.select("a")[i]["href"].strip()}" )
        i+=1

concerts = [c.split("__") for c in concerts]

print(concerts)
pprint.pprint(concerts[2:])



[['19h00-20h30 ', ' LASS ', ' MERCREDI 13 AOÃ\x9bT 2025 ', ' https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Lass'], ['20h45-22h15 ', ' SHARON SHANNON ', ' MERCREDI 13 AOÃ\x9bT 2025 ', ' https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Sharon'], ['22h30-00h00 ', ' SONERIEN DU ', ' MERCREDI 13 AOÃ\x9bT 2025 ', ' https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Sonerien'], ['19h00-20h30 ', ' CERCLE CELTIQUE AR ROUEDOÃ\x99 GLAS ', ' JEUDI 14 AOÃ\x9bT 2025 ', ' https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Cercle'], ['20h45-22h15 ', ' SOLDAT LOUIS ', ' JEUDI 14 AOÃ\x9bT 2025 ', ' https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Soldat'], ['22h30-00h00 ', " O'TRIDAL ", ' JEUDI 14 AOÃ\x9bT 2025 ', ' https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#OTridal'], ['19h00-20h30 ', ' BATTERIE FANFARE ECHO DU MONT BLANC ', ' VENDREDI 15 AOÃ\x9bT 2025 ', ' https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Fanfare'], ['20h45-22h15 ', ' 

In [ ]:
# Get constants for OA API
access_token = oa.getToken()

timezone_paris = pytz.timezone('Europe/Paris')
locationUid= get_or_create_oa_location("Quai Carnot, Concarneau",
                                        access_token,
                                        oa.public_key,
                                        f"{config.OA_API_URL}/locations")


INFO:script.libs.getOaLocation:- searching location for : 'Quai Carnot, Concarneau'
INFO:script.libs.getOaLocation: (optimized name for better matching:  'Quai Carnot Concarneau')
INFO:script.libs.getOaLocation:- 🎯 Location found in OA: ('Quai Carnot Quai Carnot, 29900 Concarneau, France', 100, 37681236)


In [ ]:
# TEST
# Date formatting. Replace 00h00 by 23h50 to avoid end before start
c=concerts[5]
date=f"{c[0].split("-")[1].strip().replace("00h00","23h50")} {c[2].strip().replace("AOÃ\x9bT", "aout")}"
print(date)
datetz=dateparser.parse(date).astimezone(timezone_paris).isoformat()
print(datetz)


23h50 JEUDI 14 aout 2025
2025-08-14T23:50:00+02:00


In [ ]:
# Create the concerts data structure based on OA formatting
concertsD= []
for index,c in enumerate(concerts):
    datedebut=dparse(f"{c[0].split("-")[0].strip().replace("00h00","23h50")} {c[2].strip().replace("AOÃ\x9bT", "aout")}").astimezone(timezone_paris).isoformat()
    datefin=dparse(f"{c[0].split("-")[1].strip().replace("00h00","23h50")} {c[2].strip().replace("AOÃ\x9bT", "aout")}").astimezone(timezone_paris).isoformat()
    concertsD.append({
        "uid-externe": "manual_scrap-filets-bleus-" + slugify(f"{c[1].strip()}"),
        "title": { "fr": f"{c[1].strip().capitalize()}"},
        "description": { "fr": f"Concert de {c[1].strip().capitalize()} - Scène Armor"},
        "longDescription": f"Concert de {c[1].strip().capitalize()} - Scène Armor",
        "timings": [
            {
                "begin": datedebut,
                "end": datefin
            }
        ],
        "lien": c[3].strip(),
        "locationUid": locationUid,
        "keywords": ['filets bleus', 'concert']
    }
    )

pprint.pprint(concertsD)

[{'description': {'fr': 'Concert de Lass - Scène Armor'},
  'keywords': ['filets bleus', 'concert'],
  'lien': 'https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Lass',
  'locationUid': 37681236,
  'longDescription': 'Concert de Lass - Scène Armor',
  'timings': [{'begin': '2025-08-13T19:00:00+02:00',
               'end': '2025-08-13T20:30:00+02:00'}],
  'title': {'fr': 'Lass'},
  'uid-externe': 'manual_scrap-filets-bleus-lass'},
 {'description': {'fr': 'Concert de Sharon shannon - Scène Armor'},
  'keywords': ['filets bleus', 'concert'],
  'lien': 'https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Sharon',
  'locationUid': 37681236,
  'longDescription': 'Concert de Sharon shannon - Scène Armor',
  'timings': [{'begin': '2025-08-13T20:45:00+02:00',
               'end': '2025-08-13T22:15:00+02:00'}],
  'title': {'fr': 'Sharon shannon'},
  'uid-externe': 'manual_scrap-filets-bleus-sharon-shannon'},
 {'description': {'fr': 'Concert de Sonerien du - Scène Armor'},
  'keyw

In [ ]:
# POST the events to OA API
for event in concertsD:
    print(event)
    print (event.get("title"))
    response = HttpRequests.create_event(access_token,
                                        event,
    )
    print(response)

{'uid-externe': 'manual_scrap-filets-bleus-lass', 'title': {'fr': 'Lass'}, 'description': {'fr': 'Concert de Lass - Scène Armor'}, 'longDescription': 'Concert de Lass - Scène Armor', 'timings': [{'begin': '2025-08-13T19:00:00+02:00', 'end': '2025-08-13T20:30:00+02:00'}], 'lien': 'https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Lass', 'locationUid': 37681236, 'keywords': ['filets bleus', 'concert']}
{'fr': 'Lass'}


INFO:root:New event: "Lass"                         https://openagenda.com/kerlandrier/contribute/event/30309196


{'success': True, 'event': {'uid': 30309196, 'slug': 'lass-5285867', 'private': False, 'timezone': 'Europe/Paris', 'draft': False, 'updatedAt': '2025-08-04T11:51:56.000Z', 'createdAt': '2025-08-04T11:51:55.592Z', 'creatorUid': 28551670, 'ownerUid': 28551670, 'agendaUid': 44891982, 'fileKey': 'bea62552c5b741ba94fc95565c5b62f0', 'attendanceMode': 1, 'onlineAccessLink': None, 'locationUid': 37681236, 'image': None, 'imageCredits': None, 'title': {'fr': 'Lass'}, 'description': {'fr': 'Concert de Lass - Scène Armor'}, 'longDescription': {'fr': 'Concert de Lass - Scène Armor'}, 'keywords': {'0': ['filets bleus'], '1': ['concert']}, 'conditions': {}, 'age': {'min': None, 'max': None}, 'registration': [], 'accessibility': {'hi': False, 'ii': False, 'vi': False, 'mi': False, 'pi': False}, 'timings': [{'begin': '2025-08-13T19:00:00.000+02:00', 'end': '2025-08-13T20:30:00.000+02:00'}], 'status': 1, 'links': [], 'location': {'uid': 37681236, 'setUid': None, 'slug': 'quai-carnot-29900-concarneau-fr

INFO:root:New event: "Sharon shannon"                         https://openagenda.com/kerlandrier/contribute/event/72813168


{'success': True, 'event': {'uid': 72813168, 'slug': 'sharon-shannon-4896953', 'private': False, 'timezone': 'Europe/Paris', 'draft': False, 'updatedAt': '2025-08-04T11:51:56.021Z', 'createdAt': '2025-08-04T11:51:56.021Z', 'creatorUid': 28551670, 'ownerUid': 28551670, 'agendaUid': 44891982, 'fileKey': 'b1b54ff710ce4f8bafffce6d16a180f5', 'attendanceMode': 1, 'onlineAccessLink': None, 'locationUid': 37681236, 'image': None, 'imageCredits': None, 'title': {'fr': 'Sharon shannon'}, 'description': {'fr': 'Concert de Sharon shannon - Scène Armor'}, 'longDescription': {'fr': 'Concert de Sharon shannon - Scène Armor'}, 'keywords': {'0': ['filets bleus'], '1': ['concert']}, 'conditions': {}, 'age': {'min': None, 'max': None}, 'registration': [], 'accessibility': {'hi': False, 'ii': False, 'vi': False, 'mi': False, 'pi': False}, 'timings': [{'begin': '2025-08-13T20:45:00.000+02:00', 'end': '2025-08-13T22:15:00.000+02:00'}], 'status': 1, 'links': [], 'location': {'uid': 37681236, 'setUid': None, 

...

{'success': True, 'event': {'uid': 23753861, 'slug': 'batterie-fanfare-echo-du-mont-blanc-7367794', 'private': False, 'timezone': 'Europe/Paris', 'draft': False, 'updatedAt': '2025-08-04T11:51:58.316Z', 'createdAt': '2025-08-04T11:51:58.316Z', 'creatorUid': 28551670, 'ownerUid': 28551670, 'agendaUid': 44891982, 'fileKey': '7f4586f90d3c450e947a8ccd07ccff50', 'attendanceMode': 1, 'onlineAccessLink': None, 'locationUid': 37681236, 'image': None, 'imageCredits': None, 'title': {'fr': 'Batterie fanfare echo du mont blanc'}, 'description': {'fr': 'Concert de Batterie fanfare echo du mont blanc - Scène Armor'}, 'longDescription': {'fr': 'Concert de Batterie fanfare echo du mont blanc - Scène Armor'}, 'keywords': {'0': ['filets bleus'], '1': ['concert']}, 'conditions': {}, 'age': {'min': None, 'max': None}, 'registration': [], 'accessibility': {'hi': False, 'ii': False, 'vi': False, 'mi': False, 'pi': False}, 'timings': [{'begin': '2025-08-15T19:00:00.000+02:00', 'end': '2025-08-15T20:30:00.00

INFO:root:New event: "Boulevard des airs"                         https://openagenda.com/kerlandrier/contribute/event/18385457


{'success': True, 'event': {'uid': 18385457, 'slug': 'boulevard-des-airs-2639522', 'private': False, 'timezone': 'Europe/Paris', 'draft': False, 'updatedAt': '2025-08-04T11:51:59.000Z', 'createdAt': '2025-08-04T11:51:58.769Z', 'creatorUid': 28551670, 'ownerUid': 28551670, 'agendaUid': 44891982, 'fileKey': 'ecc1ce0c7f3a4fc1a77cd0f1511b00af', 'attendanceMode': 1, 'onlineAccessLink': None, 'locationUid': 37681236, 'image': None, 'imageCredits': None, 'title': {'fr': 'Boulevard des airs'}, 'description': {'fr': 'Concert de Boulevard des airs - Scène Armor'}, 'longDescription': {'fr': 'Concert de Boulevard des airs - Scène Armor'}, 'keywords': {'0': ['filets bleus'], '1': ['concert']}, 'conditions': {}, 'age': {'min': None, 'max': None}, 'registration': [], 'accessibility': {'hi': False, 'ii': False, 'vi': False, 'mi': False, 'pi': False}, 'timings': [{'begin': '2025-08-15T20:45:00.000+02:00', 'end': '2025-08-15T22:15:00.000+02:00'}], 'status': 1, 'links': [], 'location': {'uid': 37681236, 

INFO:root:New event: "Ampouailh"                         https://openagenda.com/kerlandrier/contribute/event/90162237


{'success': True, 'event': {'uid': 90162237, 'slug': 'ampouailh-3361838', 'private': False, 'timezone': 'Europe/Paris', 'draft': False, 'updatedAt': '2025-08-04T11:51:59.343Z', 'createdAt': '2025-08-04T11:51:59.343Z', 'creatorUid': 28551670, 'ownerUid': 28551670, 'agendaUid': 44891982, 'fileKey': 'd0327899901445a7b21e5e8b3b972417', 'attendanceMode': 1, 'onlineAccessLink': None, 'locationUid': 37681236, 'image': None, 'imageCredits': None, 'title': {'fr': 'Ampouailh'}, 'description': {'fr': 'Concert de Ampouailh - Scène Armor'}, 'longDescription': {'fr': 'Concert de Ampouailh - Scène Armor'}, 'keywords': {'0': ['filets bleus'], '1': ['concert']}, 'conditions': {}, 'age': {'min': None, 'max': None}, 'registration': [], 'accessibility': {'hi': False, 'ii': False, 'vi': False, 'mi': False, 'pi': False}, 'timings': [{'begin': '2025-08-15T22:30:00.000+02:00', 'end': '2025-08-15T23:50:00.000+02:00'}], 'status': 1, 'links': [], 'location': {'uid': 37681236, 'setUid': None, 'slug': 'quai-carnot

In [ ]:
# CORRECTION
# Update the events with online access link and attendance mode
for concert in concertsD:
    print(concert.get("uid-externe"))
    event= HttpRequests.search_events(oa.public_key, concert.get("title").get("fr")).get("events")[0]
    # pprint.pprint(event)
    eventUID= event.get("uid")
    print(eventUID)
    concert["onlineAccessLink"] = concert.get("lien")
    concert["attendanceMode"] = 3  # 1=offline, 2=online, 3=hybrid
    HttpRequests.patch_event(
        access_token,
        eventUID,
        concert
    )
    print("--------")

manual_scrap-filets-bleus-lass
30309196


INFO:root:{'success': True, 'event': {'uid': 30309196, 'slug': 'lass-5285867', 'private': 0, 'timezone': 'Europe/Paris', 'draft': 0, 'updatedAt': '2025-08-04T13:06:15.000Z', 'createdAt': '2025-08-04T11:51:56.000Z', 'creatorUid': 28551670, 'ownerUid': 28551670, 'agendaUid': 44891982, 'fileKey': 'bea62552c5b741ba94fc95565c5b62f0', 'attendanceMode': 3, 'onlineAccessLink': 'https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Lass', 'locationUid': 37681236, 'image': None, 'imageCredits': None, 'title': {'fr': 'Lass'}, 'description': {'fr': 'Concert de Lass - Scène Armor'}, 'longDescription': {'fr': 'Concert de Lass - Scène Armor', 'en': 'Concert de Lass - Scène Armor'}, 'keywords': {'0': ['filets bleus'], '1': ['concert']}, 'conditions': {}, 'age': {'min': None, 'max': None}, 'registration': [], 'accessibility': {'hi': False, 'ii': False, 'vi': False, 'mi': False, 'pi': False}, 'timings': [{'begin': '2025-08-13T19:00:00.000+02:00', 'end': '2025-08-13T20:30:00.000+02:00'}], 'extIds': N

--------
manual_scrap-filets-bleus-sharon-shannon
72813168


INFO:root:{'success': True, 'event': {'uid': 72813168, 'slug': 'sharon-shannon-4896953', 'private': 0, 'timezone': 'Europe/Paris', 'draft': 0, 'updatedAt': '2025-08-04T13:06:16.000Z', 'createdAt': '2025-08-04T11:51:56.000Z', 'creatorUid': 28551670, 'ownerUid': 28551670, 'agendaUid': 44891982, 'fileKey': 'b1b54ff710ce4f8bafffce6d16a180f5', 'attendanceMode': 3, 'onlineAccessLink': 'https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Sharon', 'locationUid': 37681236, 'image': None, 'imageCredits': None, 'title': {'fr': 'Sharon shannon'}, 'description': {'fr': 'Concert de Sharon shannon - Scène Armor'}, 'longDescription': {'fr': 'Concert de Sharon shannon - Scène Armor', 'en': 'Concert de Sharon shannon - Scène Armor'}, 'keywords': {'0': ['filets bleus'], '1': ['concert']}, 'conditions': {}, 'age': {'min': None, 'max': None}, 'registration': [], 'accessibility': {'hi': False, 'ii': False, 'vi': False, 'mi': False, 'pi': False}, 'timings': [{'begin': '2025-08-13T20:45:00.000+02:00', '

--------
manual_scrap-filets-bleus-sonerien-du
27698426


...

--------
manual_scrap-filets-bleus-boulevard-des-airs
18385457


INFO:root:{'success': True, 'event': {'uid': 18385457, 'slug': 'boulevard-des-airs-2639522', 'private': 0, 'timezone': 'Europe/Paris', 'draft': 0, 'updatedAt': '2025-08-04T13:06:23.000Z', 'createdAt': '2025-08-04T11:51:59.000Z', 'creatorUid': 28551670, 'ownerUid': 28551670, 'agendaUid': 44891982, 'fileKey': 'ecc1ce0c7f3a4fc1a77cd0f1511b00af', 'attendanceMode': 3, 'onlineAccessLink': 'https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Boulevard', 'locationUid': 37681236, 'image': None, 'imageCredits': None, 'title': {'fr': 'Boulevard des airs'}, 'description': {'fr': 'Concert de Boulevard des airs - Scène Armor'}, 'longDescription': {'fr': 'Concert de Boulevard des airs - Scène Armor', 'en': 'Concert de Boulevard des airs - Scène Armor'}, 'keywords': {'0': ['filets bleus'], '1': ['concert']}, 'conditions': {}, 'age': {'min': None, 'max': None}, 'registration': [], 'accessibility': {'hi': False, 'ii': False, 'vi': False, 'mi': False, 'pi': False}, 'timings': [{'begin': '2025-08-15

--------
manual_scrap-filets-bleus-ampouailh
90162237


INFO:root:{'success': True, 'event': {'uid': 90162237, 'slug': 'ampouailh-3361838', 'private': 0, 'timezone': 'Europe/Paris', 'draft': 0, 'updatedAt': '2025-08-04T13:06:24.022Z', 'createdAt': '2025-08-04T11:51:59.000Z', 'creatorUid': 28551670, 'ownerUid': 28551670, 'agendaUid': 44891982, 'fileKey': 'd0327899901445a7b21e5e8b3b972417', 'attendanceMode': 3, 'onlineAccessLink': 'https://www.festivaldesfiletsbleus.bzh/artistes_2025.html#Ampouailh', 'locationUid': 37681236, 'image': None, 'imageCredits': None, 'title': {'fr': 'Ampouailh'}, 'description': {'fr': 'Concert de Ampouailh - Scène Armor'}, 'longDescription': {'fr': 'Concert de Ampouailh - Scène Armor', 'en': 'Concert de Ampouailh - Scène Armor'}, 'keywords': {'0': ['filets bleus'], '1': ['concert']}, 'conditions': {}, 'age': {'min': None, 'max': None}, 'registration': [], 'accessibility': {'hi': False, 'ii': False, 'vi': False, 'mi': False, 'pi': False}, 'timings': [{'begin': '2025-08-15T22:30:00.000+02:00', 'end': '2025-08-15T23:5

--------
